In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pydicom
from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
import tensorflow as tf
import time

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from keras.datasets import mnist
# https://keras.io/datasets/

Using TensorFlow backend.


In [3]:
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [4]:
test_df = pd.read_csv('data/test_dataset.csv')
train_df = pd.read_csv('data/train_dataset.csv')

- SHUFFLE DATA FRAME

In [5]:
train_df=train_df.sample(frac=1)

- Get Targets in a series

In [6]:
Class = train_df.Target

- Get Image Paths from CSV column(patientID)

In [7]:
image_path = []
for i in train_df.patientId:
    image_path.append('images/%s.dcm' % i)

len(image_path)

1000

- Read images with pydicom

In [8]:
images = []
for i in image_path:
    images.append(pydicom.read_file(i))

- Get Pixel array with (pydicom.pixel_array)

In [9]:
pixel_s = []
for i in range(len(images)):
    pixel_s.append(images[i].pixel_array)

- Scale Images
- scale = 1.00% = 1054x1054

- How many unique IDs/x-rays did we load

In [10]:
len(pixel_s)

1000

- Scale Images
- Scale = %1.00= 1054 X 1054

In [11]:
scaled = []
image_scale = 1
for i in range(len(pixel_s)):
    scaled.append(rescale(pixel_s[i],scale=image_scale,mode='reflect',multichannel=False))

C:\Users\James\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [12]:
width = len(scaled[0])
height = len(scaled[1])
print("IMAGE SIZE: ",height, width)

IMAGE SIZE:  1024 1024


- Make giant NUMPY ARRAY of all images and all pixels

In [13]:
X_train=np.array(scaled)
X_train.shape

(1000, 1024, 1024)

- Class variable holds our target values
- .values takes pandas.series => numpy array

In [14]:
y_train = Class.values

- reshape to be [samples][pixels][width][height]


In [15]:
X_train = X_train.reshape(X_train.shape[0], width, height,1 ).astype('float32') # for the Convnet

In [16]:
X_train.shape

(1000, 1024, 1024, 1)

- normalize inputs from 0-255 to 0-1

In [17]:
X_train = X_train / 255

- one hot encode outputs

In [18]:
y_train = to_categorical(y_train)
y_train.shape

(1000, 2)

In [19]:
int(time.time())

1540300850

In [20]:
num_classes = y_train.shape[1]
num_classes

2

In [21]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(width, height, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# save logs for tensorboard
tensorboard = TensorBoard(log_dir=f'logs/{int(time.time())}_{width}x{height}')

In [22]:
filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

In [ ]:
# Fit (train) the model
model.fit(
    X_train,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2,
    validation_split = .2,
    callbacks=[tensorboard, checkpoint]
)

Train on 800 samples, validate on 200 samples
Epoch 1/10


In [ ]:
#tensorboard --logdir=logs
#http://DESKTOP-D2C2TFC:6006